In [ ]:
import mysql.connector
import pandas as pd
from mysql.connector import Error

def execute_query(query, con, cursor, values=None):
    try:
        if values:
            cursor.executemany(query, values)
        else:
            cursor.execute(query)
    except Error as e:
        print(f"Error: {e}")

def create_database_and_table():
    try:
        con = mysql.connector.connect(host='localhost', user='root', password='04101998')
        cursor = con.cursor()

        # Create database and table
        queries = [
            'DROP DATABASE IF EXISTS redbus',
            'CREATE DATABASE redbus',
            'USE redbus',
            '''
            CREATE TABLE bus_routes (
                id INT AUTO_INCREMENT PRIMARY KEY,
                route_name TEXT,
                route_link TEXT,
                busname TEXT,
                bustype TEXT,
                departing_time DATETIME,
                duration TEXT,
                reaching_time DATETIME,
                star_rating FLOAT DEFAULT 0.0,
                price DECIMAL(10, 2),
                seats_available INT
            )
            '''
        ]
        for query in queries:
            execute_query(query, con, cursor)
        print("Database and Table Created")

    finally:
        if con.is_connected():
            cursor.close()
            con.close()

def insert_data_from_excel():
    try:
        con = mysql.connector.connect(host='localhost', user='root', password='04101998', database='redbus')
        cursor = con.cursor()

        # Read data from Excel
        df = pd.read_excel('output_redbus.xlsx')
        data = df.values.tolist()

        query = '''
        INSERT INTO bus_routes (
            route_name, route_link, busname, bustype, departing_time, duration,
            reaching_time, star_rating, price, seats_available
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''

        execute_query(query, con, cursor, data)

        con.commit()
        print("Values inserted successfully")

    finally:
        if con.is_connected():
            cursor.close()
            con.close()

if __name__ == "__main__":
    create_database_and_table()
    insert_data_from_excel()
